In [1]:
from distributions.distribution_by_sequence import DistributionBySequence
from distributions.distribution import Distribution
from distributions.sequence import Sequence
from algorithms.semi_bandit_exp3 import SemiBanditExp3
from algorithms.full_bandit_exp3 import FullBanditExp3
from algorithms.semi_bandit_ftrl import SemiBanditFTRL
from algorithms.semi_bandit_ftrl_inv import SemiBanditFTRLInv
from algorithms.full_bandit_exp3_inv import FullBanditExp3Inv
from algorithms.real_lin_exp3 import RealLinExp3
from algorithms.uniform_random import UniformRandom
from algorithms.non_contextual_exp3 import NonContextualExp3
from algorithms.one_per_context import OnePerContext
from experiment_manager.experiment_manager import ExperimentManager

from algorithms.shortest_path import ShortestPath
from algorithms.bubeck import Bubeck
from algorithms.one_per_context_bubeck import OnePerContextBubeck

from misc.tensor_helpers import *

from distributions.actionsets.msets import MSets

from distributions.contexts.binary_context import BinaryContext
from distributions.thetas.single_hole import SingleHole
from distributions.thetas.independent_bernoulli import IndependentBernoulli



In [2]:
def get_dist(rng, d, K, m):
    p = np.zeros((d, K)) + 0.5
    for i in range(d):
        placed_already = []
        while len(placed_already) < m:

            index = rng.integers(K)
            if index not in placed_already:
                placed_already.append(index)
                p[i, index] = 0.4
    
    return IndependentBernoulli(d, K, p)


In [3]:
rng = np.random.default_rng()
length = 10000
d = 4
number_of_ones = 2
K = 3
m  = 1
actionset = MSets(K, m)

epsilon = 0.25 * np.min([np.sqrt(K / length), 1])
p = np.zeros((d, K)) + 0.5
for i in range(d):
    p[i, 0] = 0.45

dist_lower_bound = Distribution(BinaryContext(d, number_of_ones), get_dist(rng, d, K, m), actionset)


seq = dist_lower_bound.generate(length, rng, rng)
algo = Bubeck()
algo.set_constants(rng, seq)
print("sigma:", seq.sigma, "m:", seq.m, "beta:", algo.beta, "gamma:", algo.gamma, "eta", algo.eta)
#algo.gamma = 0.06887868239885406
#algo.eta = 0.0010063304877787227
loss, _, _, _ = algo.run_on_sequence(rng, seq)
loss_of_optimal_policy, _, _ = seq.find_optimal_policy()
print("Bubeck", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = OnePerContextBubeck()
algo.set_constants(rng, seq)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
print("OnePerContextBubeck", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = ShortestPath()
algo.set_constants(rng, seq)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
print("ShortestPath", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = SemiBanditFTRLInv()
algo.set_constants(rng, seq)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
print("SemiBanditFTRLInv", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = UniformRandom()
algo.set_constants(rng, seq)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
print("UniformRandom", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = RealLinExp3()
algo.set_constants(rng, seq)
algo.gamma = np.sqrt(1 / seq.length)
algo.eta = np.sqrt(np.log(len(seq.actionset.actionset)) / (seq.d * len(seq.actionset.actionset) * seq.length))
loss, _, _, _ = algo.run_on_sequence(rng, seq)
print("RealLinExp3NoLog", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = RealLinExp3()
algo.set_constants(rng, seq)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
print("RealLinExp3", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = OnePerContext()
algo.set_constants(rng, seq)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
context_algo = algo.context_algos[list(algo.context_algos.keys())[0]]
print("OnePerContext", loss - loss_of_optimal_policy, algo.gamma, algo.eta)

algo = FullBanditExp3Inv()
algo.set_constants(rng, seq)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
print("FullBanditExp3Inv", loss - loss_of_optimal_policy, algo.gamma, algo.eta)



sigma: 1.4142135623730951 m: 1 beta: 0.4999999999999999 gamma: 0.017320508075688773 eta 0.0060514799530586175
Bubeck 457.0 0.017320508075688773 0.0060514799530586175
OnePerContextBubeck 470.0 None None
ShortestPath 552.0 0.05256521769756932 0.0029202898720871845
SemiBanditFTRLInv 710.0 0.21538207996457961 0.0006740662842713811
UniformRandom 731.0 None None
RealLinExp3NoLog 249.0 0.01 0.0030257399765293088
RealLinExp3 608.0 0.035603477744141666 0.0008498439389189096
OnePerContext 550.0 None None
FullBanditExp3Inv 981.0 0.15583524465641818 0.0007044865450771258


In [4]:
algo = SemiBanditFTRLInv()
algo.set_constants(rng, seq)

seq.reset()
context, _, _, done = seq.get_next(None)

losses = []
probability_array = []
action_array = []
while not done:
    probabilities = algo.get_policy(context)
    probability_array.append(probabilities)

    print("Start next")
    print(context, probabilities)
    action_index = rng.choice(np.arange(seq.actionset.number_of_actions), p=probabilities)
    print(action_index, seq.actionset[action_index])
    action_array.append(action_index)

    print("\n")
    next_context, loss, loss_vec, done = seq.get_next(seq.actionset[action_index])
    print(loss, loss_vec)
    loss_vec[~seq.actionset[action_index]] = 0
    print(loss, loss_vec)
    if algo.full_bandit:
        algo.observe_loss(loss, context, action_index)
    else:
        algo.observe_loss_vec(loss_vec, context, action_index)

    print(algo.theta_estimate)
    print("\n\n")
    losses.append(loss)
    context = next_context

    if seq.current_index == 100:
        1/0


Start next
[1. 0. 0. 1.] [0.33333333 0.33333333 0.33333333]
1 [False  True False]


1.0 [2. 1. 0.]
1.0 [0. 1. 0.]
[[ 0.  6.  0.]
 [ 0. -3.  0.]
 [ 0. -3.  0.]
 [ 0.  6.  0.]]



Start next
[1. 1. 0. 0.] [0.33350957 0.33298086 0.33350957]
0 [ True False False]


2.0 [2. 0. 1.]
2.0 [2. 0. 0.]
[[11.98891127  6.          0.        ]
 [11.99841472 -3.          0.        ]
 [-5.9920736  -3.          0.        ]
 [-6.00157705  6.          0.        ]]



Start next
[1. 0. 0. 1.] [0.3333334  0.33227554 0.33439107]
2 [False False  True]


0.0 [0. 1. 0.]
0.0 [0. 0. 0.]
[[11.98891127  6.          0.        ]
 [11.99841472 -3.          0.        ]
 [-5.9920736  -3.          0.        ]
 [-6.00157705  6.          0.        ]]



Start next
[0. 1. 1. 0.] [0.33227554 0.33439219 0.33333228]
2 [False False  True]


1.0 [2. 0. 1.]
1.0 [0. 0. 1.]
[[11.98891127  6.         -2.9999983 ]
 [11.99841472 -3.          5.99051616]
 [-5.9920736  -3.          6.000016  ]
 [-6.00157705  6.         -2.99049847]]





ZeroDivisionError: division by zero